In [2]:
""" In this Notebook I shall investigate using Osborne's PIGEBaQ method on 
the test data obtained from my permutation algorithm. We shall see if the results agree, 
and learn a bit about the challenges of using this approach."""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

df = pd.read_csv("/Users/jacob/Documents/4YP data/ONET_18-10-21/test.csv") #This is all the data I will need
df.drop(columns=['O*NET-SOC Code', 'Title'], inplace=True)
df.head()

,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,Learning Strategies,Monitoring,...,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources,Auto label value
0,4.75,4.88,4.38,4.88,3.62,1.12,4.75,4.75,3.88,5.50,...,0.0,1.00,5.75,5.38,5.12,4.75,5.50,4.75,5.38,0.0
1,4.25,4.00,4.25,4.12,3.12,1.88,4.12,3.88,3.75,4.00,...,0.0,1.50,4.00,4.00,4.00,3.88,3.12,2.62,4.00,NaN
2,4.00,4.00,3.88,4.00,2.50,1.12,4.00,3.62,3.25,4.00,...,0.0,2.12,3.75,3.00,3.12,3.75,3.38,3.25,3.88,NaN
3,4.00,4.12,3.88,4.12,3.25,0.62,4.12,4.12,3.25,4.25,...,0.0,1.00,4.00,3.12,3.75,3.88,3.62,2.62,3.88,NaN
4,4.25,4.12,3.88,4.12,3.12,1.50,4.25,4.12,3.50,4.25,...,0.0,1.38,4.00,3.75,3.75,3.75,3.75,2.75,3.88,NaN


In [3]:
#Lets now drop the unseen observations to leave just the training data:

df1 = df.dropna()
df1.reset_index(drop=True,inplace=True)
df1.head()

,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,Learning Strategies,Monitoring,...,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources,Auto label value
0,4.75,4.88,4.38,4.88,3.62,1.12,4.75,4.75,3.88,5.50,...,0.00,1.00,5.75,5.38,5.12,4.75,5.50,4.75,5.38,0.0
1,4.00,3.96,3.67,3.96,3.17,0.46,4.08,3.66,3.38,4.13,...,0.08,1.96,3.63,3.46,3.67,3.88,3.33,3.17,3.79,0.0
2,4.12,4.00,4.00,4.00,2.88,0.75,4.00,3.88,3.88,4.12,...,0.00,1.75,3.88,3.12,3.38,3.88,3.25,3.12,4.00,0.0
3,4.12,4.25,4.00,4.12,3.12,2.00,4.12,3.88,3.75,4.00,...,0.00,1.62,4.12,3.75,3.88,4.00,3.12,2.38,4.38,0.0
4,4.06,3.94,3.75,3.88,3.06,0.44,4.00,3.31,2.56,3.12,...,0.00,1.81,3.62,2.44,2.44,2.94,2.19,1.56,2.25,1.0


In [4]:
#We first need to model the input variables of our data. We shall assume they are gaussian (non-mixture)

mu = list(df.mean(axis='index'))
del mu[-1]
m = len(mu)
print(m)

n = len(df1.index)
print(n)

np_mu = np.array(mu)
print(np_mu)
#We have our vector of means (mx1)

35
63
[3.64368843 3.54613975 3.29475372 3.45728522 2.56115693 1.50342497
 3.57       3.19258877 2.84367698 3.37768614 3.10325315 3.14676976
 2.76437572 2.564937   2.91871707 2.85216495 3.15989691 1.73046964
 1.00459336 1.07781214 0.35969072 0.81539519 2.28172967 1.7741008
 1.05408935 1.56753723 1.02463918 2.17970218 3.20827033 2.63945017
 2.62533792 3.0472394  1.34610538 1.48531501 2.58720504]


In [5]:
#Now lets get our covariance matrix:

df2 = df1.drop(columns=['Auto label value']) #This dataframe removes the output of our GP so the covariance isn't effected

df_cov = df2.cov()
df_cov.head()

np_cov = df_cov.to_numpy()
print(np_cov)
#We have our covariance matrix (mxm)

[[0.76444782 0.58130456 0.69916557 ... 0.45318431 0.3671946  0.33895143]
 [0.58130456 0.56572002 0.57766101 ... 0.3614607  0.27286907 0.31405394]
 [0.69916557 0.57766101 0.70013492 ... 0.38176979 0.30487363 0.32323369]
 ...
 [0.45318431 0.3614607  0.38176979 ... 1.31053507 1.10093914 0.76044068]
 [0.3671946  0.27286907 0.30487363 ... 1.10093914 1.00709621 0.6898353 ]
 [0.33895143 0.31405394 0.32323369 ... 0.76044068 0.6898353  0.71434444]]


In [6]:
print(np.shape(np_mu))
print(np.shape(np_cov))

(35,)
(35, 35)


In [7]:
#Now lets focus on the matrix of lengthscales and the output variance of the kernel:

output_var = 125
lengthscale = 8  #both values estimated while get_params method continues to not work
np_lengthscale = lengthscale*np.identity(m)



In [8]:
#We now have all the ingredients we need to make z_n (for each observation):
#Lets begin by just focusing on the first observation, and then creating a function for this
import math

x = np.array(df2.iloc[[0]])

element1 = (output_var*math.sqrt(np.linalg.det(np_lengthscale)))/(math.sqrt(np.linalg.det(np_lengthscale+np_cov)))
#print(element1)
element2 = np.linalg.inv(np_lengthscale+np_cov)
#print(element2)
element3 = np.transpose(np_mu - x)
#print(element3)
#temp = np.matmul(element2,-1*element3)
element4 = -0.5*np.matmul(np.transpose(-1*element3),np.matmul(element2,-1*element3))
#print(element4)

z = element1*math.exp(element4.item())*np.matmul(element2,element3)
print(z)

[[-0.00860548]
 [-0.06894903]
 [-0.01170304]
 [-0.0725187 ]
 [-0.02920896]
 [ 0.22937517]
 [-0.03208581]
 [-0.07919282]
 [-0.01842026]
 [-0.22136611]
 [-0.0328072 ]
 [-0.20914795]
 [-0.1989016 ]
 [-0.18529898]
 [-0.02611171]
 [ 0.02813506]
 [-0.15039913]
 [-0.20639342]
 [ 0.1156125 ]
 [ 0.03784764]
 [ 0.0399778 ]
 [ 0.12448286]
 [ 0.09360752]
 [-0.02798726]
 [ 0.09913313]
 [ 0.20658357]
 [ 0.11353483]
 [ 0.17739136]
 [-0.26150596]
 [-0.26995348]
 [-0.20238852]
 [-0.16396686]
 [-0.48638356]
 [-0.36933394]
 [-0.31257482]]


In [9]:
#Lets now create a function that does this for each observation:

def compute_zn(index):
    x = np.array(df2.iloc[[index]])

    element1 = (output_var*math.sqrt(np.linalg.det(np_lengthscale)))/(math.sqrt(np.linalg.det(np_lengthscale+np_cov)))
    element2 = np.linalg.inv(np_lengthscale+np_cov)
    element3 = np.transpose(np_mu - x)
    element4 = -0.5*np.matmul(np.transpose(-1*element3),np.matmul(element2,-1*element3))

    zn = element1*math.exp(element4.item())*np.matmul(element2,element3)
    return zn


In [10]:
Z = [0]*n
for i in range(0,n):
    Z[i] = compute_zn(i)

Z = np.array(Z)
Z = Z.reshape(m,n)
print(Z)

[[-0.00860548 -0.06894903 -0.01170304 ...  0.76190741  1.61546301
   0.19852163]
 [ 0.05586835 -0.70885447 -1.11877989 ...  0.14882637  1.74613009
   0.50923952]
 [ 0.394528   -0.20829026 -0.29004295 ...  0.91202445 -0.87630489
  -1.100605  ]
 ...
 [ 0.77157932  0.44761931 -0.40216863 ...  2.04667063  0.72953125
   1.09449997]
 [ 1.97317995  0.32414552  1.364847   ... -0.511011    1.39508738
   0.54003225]
 [ 1.59337505  0.58907687  1.02868886 ... -0.28181533  0.23086294
   1.1566362 ]]


In [11]:
#We now need to find K, which is a matrix of the kernel outputs of the training data.
X = df2.to_numpy()
X_norm = np.sum(X ** 2, axis = -1)
K = output_var * np.exp(-(1/lengthscale) * (X_norm[:,None] + X_norm[None,:] - 2 * np.dot(X, X.T)))
print(K)
np.shape(K)

#I don't trust this code

[[1.25000000e+02 8.79337705e-01 2.96228619e-01 ... 8.03007993e-08
  4.34522338e-10 3.73584772e-11]
 [8.79337705e-01 1.25000000e+02 4.59005104e+01 ... 5.24635696e-02
  9.47215570e-03 1.49783243e-03]
 [2.96228619e-01 4.59005104e+01 1.25000000e+02 ... 1.41156134e-02
  7.43236429e-03 4.24277107e-04]
 ...
 [8.03007993e-08 5.24635696e-02 1.41156134e-02 ... 1.25000000e+02
  7.05817366e-01 1.19479981e+01]
 [4.34522338e-10 9.47215570e-03 7.43236429e-03 ... 7.05817366e-01
  1.25000000e+02 1.66623426e+01]
 [3.73584772e-11 1.49783243e-03 4.24277107e-04 ... 1.19479981e+01
  1.66623426e+01 1.25000000e+02]]


(63, 63)

In [26]:
# I do not trust the code that I've been presented with. I shall create my own function for computing the covariance matrix:

X = df2.to_numpy() #Each row is an abservation, each column is a variable

def compute_cov(x,y,var,l):  #x and y are each full observations (vectors)
    d = 0
    m = len(x)
    for i in range(m):
        d += (x[i]-y[i])**2
    val = var*np.exp(-0.5*d/(l**2))
    return val
#This function takes in two observations and outputs a scalar


num = df2.shape[0]
K = np.empty([num,num])

#Now lets start filling this matrix:

K[0,0] = compute_cov(X[0,:],X[0,:],output_var,lengthscale)
#K

for i in range(num):
    for j in range(num):
        K[i,j] = compute_cov(X[i,:],X[j,:],output_var,lengthscale)

K

#Which of these methods should I trust?

array([[125.        ,  91.69863442,  85.67020883, ...,  33.29646507,
         24.02862357,  20.61234771],
       [ 91.69863442, 125.        , 117.41314791, ...,  76.88546618,
         69.08459071,  61.56297362],
       [ 85.67020883, 117.41314791, 125.        , ...,  70.82871649,
         68.04536906,  56.89593478],
       ...,
       [ 33.29646507,  76.88546618,  70.82871649, ..., 125.        ,
         90.44746691, 107.94041504],
       [ 24.02862357,  69.08459071,  68.04536906, ...,  90.44746691,
        125.        , 110.20762438],
       [ 20.61234771,  61.56297362,  56.89593478, ..., 107.94041504,
        110.20762438, 125.        ]])

In [27]:
#Final ingredient: we need a row vector of the outputs:

f = df1["Auto label value"].to_numpy()
f = f.reshape(63)
np.shape(f)

(63,)

In [28]:
#Moment of truth...

E = np.matmul(Z,np.matmul(np.linalg.inv(K),f))
print(E)

[ 1.14964646  5.42178878 -7.9328146   0.6369632   1.23118419  3.82719757
  4.20191414 -7.26342922  4.03859191 -0.31367138  2.85058822 -4.37243335
 -3.76126644  1.40708163  0.29877881 -3.57626968 -3.70770537 -2.47611244
  3.54966393 -0.6463692   1.48245035  2.43750781  0.5945215   1.64923033
  1.8976617  -0.72129151  1.07350224 -4.35600845  3.67576978 -1.65115823
  4.48369962 -2.24000178  1.55190675 -2.41752283 -0.35813421]


In [29]:
print(np.shape(Z))
print(np.shape(K))
print(np.shape(f))

(35, 63)
(63, 63)
(63,)


In [30]:
#Lets now hitch these values onto a new dataframe of skills:

output_df = pd.DataFrame({"Skills":df2.columns,"Importance":E})

In [31]:
output_df = output_df.sort_values(by="Importance")
output_df.reset_index(drop=True, inplace = True)
print(output_df)

                               Skills  Importance
0                             Writing   -7.932815
1                     Active Learning   -7.263429
2                        Coordination   -4.372433
3            Quality Control Analysis   -4.356008
4                          Persuasion   -3.761266
5             Complex Problem Solving   -3.707705
6                 Service Orientation   -3.576270
7                 Operations Analysis   -2.476112
8    Management of Material Resources   -2.417523
9                     Time Management   -2.240002
10                   Systems Analysis   -1.651158
11                    Troubleshooting   -0.721292
12                Equipment Selection   -0.646369
13  Management of Personnel Resources   -0.358134
14                         Monitoring   -0.313671
15                        Instructing    0.298779
16              Operations Monitoring    0.594522
17                           Speaking    0.636963
18                          Repairing    1.073502


In [ ]:
output_df.to_csv("PIGEBaQ_Ex.csv")

In [12]:
X_norm

array([554.0729, 330.1999, 332.6104, 378.3367, 237.0379, 243.5691,
       299.7154, 187.9249, 301.2836, 298.1673, 283.1239, 234.4784,
       219.5566, 232.146 , 215.9703, 323.7935, 310.0904, 403.8916,
       207.7635, 241.111 , 308.8178, 504.3654, 308.5041, 310.5558,
       354.0708, 380.4842, 351.2378, 244.381 , 364.0085, 188.4971,
       205.7941, 286.1248, 186.5247, 222.3067, 210.6354, 391.0291,
       138.788 , 318.1841,  97.0642, 116.4398, 104.9929,  91.0192,
       124.5191, 176.4997, 177.2292, 177.4204, 126.6923, 112.8203,
       180.6432, 129.7653, 204.0797, 123.6887, 152.7537, 135.9679,
       247.3617, 205.4042, 170.0322,  99.6029, 199.6029, 139.1711,
       232.2723, 109.019 , 133.3697])

In [13]:
np.shape(X_norm)

(63,)

In [23]:
X_norm[:,None]

array([[554.0729],
       [330.1999],
       [332.6104],
       [378.3367],
       [237.0379],
       [243.5691],
       [299.7154],
       [187.9249],
       [301.2836],
       [298.1673],
       [283.1239],
       [234.4784],
       [219.5566],
       [232.146 ],
       [215.9703],
       [323.7935],
       [310.0904],
       [403.8916],
       [207.7635],
       [241.111 ],
       [308.8178],
       [504.3654],
       [308.5041],
       [310.5558],
       [354.0708],
       [380.4842],
       [351.2378],
       [244.381 ],
       [364.0085],
       [188.4971],
       [205.7941],
       [286.1248],
       [186.5247],
       [222.3067],
       [210.6354],
       [391.0291],
       [138.788 ],
       [318.1841],
       [ 97.0642],
       [116.4398],
       [104.9929],
       [ 91.0192],
       [124.5191],
       [176.4997],
       [177.2292],
       [177.4204],
       [126.6923],
       [112.8203],
       [180.6432],
       [129.7653],
       [204.0797],
       [123.6887],
       [152.

In [16]:
X_norm[None,:]

(1, 63)

In [18]:
np.dot(X, X.T)

array([[554.0729, 422.3088, 419.1619, ..., 308.5094, 226.0056, 228.3662],
       [422.3088, 330.1999, 327.3978, ..., 250.1323, 181.6586, 186.4566],
       [419.1619, 327.3978, 332.6104, ..., 246.0862, 181.8938, 182.6163],
       ...,
       [308.5094, 250.1323, 246.0862, ..., 232.2723, 149.9388, 173.43  ],
       [226.0056, 181.6586, 181.8938, ..., 149.9388, 109.019 , 113.1337],
       [228.3662, 186.4566, 182.6163, ..., 173.43  , 113.1337, 133.3697]])

In [22]:
X_norm[:,None] + X_norm[None,:]

array([[1108.1458,  884.2728,  886.6833, ...,  786.3452,  663.0919,
         687.4426],
       [ 884.2728,  660.3998,  662.8103, ...,  562.4722,  439.2189,
         463.5696],
       [ 886.6833,  662.8103,  665.2208, ...,  564.8827,  441.6294,
         465.9801],
       ...,
       [ 786.3452,  562.4722,  564.8827, ...,  464.5446,  341.2913,
         365.642 ],
       [ 663.0919,  439.2189,  441.6294, ...,  341.2913,  218.038 ,
         242.3887],
       [ 687.4426,  463.5696,  465.9801, ...,  365.642 ,  242.3887,
         266.7394]])

In [24]:
df2.shape[0]

63

In [34]:
compute_cov(X[0,:],X[1,:],1,8)

0.7335890753394024

In [36]:
test = X[0,:]
test[1]

4.88